# <font color = "darkblue"> Introducción avanzada Julia  </font>

### <font color = "darkblue"> Estructuras o Tipos  </font>

Las estructuras, o tipos, son centrales en Julia. La instrucción `typeof` sirve para saber el tipo de estructura de una variable o algún valor. 


Un tipo es básicamente qué clase de "ente" es un valor dado, ya hemos hablado de algunos de estos, tales son: flotantes(Float), enteros(Int), cadenas de caracteres(Strings), funciones, etc.

Podemos declar el tipo de una variable mediante la sintáxis `var::Tipo = valor`(por ahora Julia sólo permite declarar el tipo de una variable dentro de funciones) esto es algo muy parecido en lo que se hace en algunos lenguajes como C.

En Julia existe una jerarquía de tipos, en la cual el tipo `Any` es el "padre" de todos los tipos, podemos verficar si un tipo es un subtipo de otro mediante el operador `<:` el cuál puede ser empleado como

In [ ]:
Float64 <: Float64 # Un tipo siempre es subtipo de sí mismo

In [ ]:
Float64 <: AbstractFloat # AbstractFloat es un tipo que es padre de todos los Float

In [ ]:
Arreglo = [1 2 3 3 2 1]

In [ ]:
typeof(Arreglo)

## <font color = "darkblue"> Funciones  </font>

**Hay tres maneras de definir una función**

In [ ]:
function f1(x)
    
    return x^2
end

En el ejemplo anterior definimos la función `f`, que depende de un argumento `(x)`, cuyo tipo no se ha especificado, es decir, **NO** hemos restringido el valor de `x`de ninguna manera. Por lo tanto, podemos usar la función `f` en objetos en los que tenga sentido "elevar al cuadrado". Esto es lo que se llama "teclear como pato" (*duck typing*).

In [ ]:
f1(2)

In [ ]:
f1("Una cadena")

La segunda manera de definir una función es en una línea, como es común ver cosas escritas en matemáticas. El caso anterior podría haber sido definido de la siguiente manera:

In [ ]:
g(x) = x^2

In [ ]:
g("Algo")

Un tercer método para definir funciones es lo que se llama funciones anónimas. Nuevamente, su construcción es usual en las matemáticas, excepto por que no tienen nombre.

In [ ]:
gg = x -> x^2  # `g` es el nombre de la función anónima

In [ ]:
gg(2)

Ahora definiremos un *segundo método* de la función `f`. Por ejemplo, queremos que si `f` se aplica a una cadena, el resultado sea un mensaje específico. Para esto, especificaremos que `x` es una cadena cualquiera usando `x::AbstractString`, lo que permitirá que usemos este método exclusivamente para cadenas.

In [ ]:
f1(x::AbstractString) = "No se leer"

In [ ]:
f1("Cadena")

In [ ]:
f1(x::Int64) = x^4

In [ ]:
f1(2)

Vale la pena notar que `f` aplicado a un vector o matriz, arroja un error. Sin embargo, podemos aplicar `f` al vector usando puntos (*dots*). Esto es lo que se llama transimisión (*broadcasting*). Esta es una notación conveniente, aunque no siempre es la más rápida. El punto es que la definición genérica de `f(x)` la podemos generalizar en vectores o matrices.

In [ ]:
v = [1 2; 3 4]

In [ ]:
f1.(v)

## <font color = "darkblue"> Estabilidad de tipo </font>


Para que quede claro la importancia de la estabilidad de tipo, hagamos un ejemplo de una función que no es estable según el tipo, y comparemos su rendimiento con una que sí lo es. 

In [ ]:
function inestable(n::Int)
    x = rem(n,2) == 0 ? n/2 : n//3 # x puede ser un Float64 o un racional
    return x^2
end

# Aquí, x sólo puede ser un Float64
function estable(n::Int)
    x = rem(n,2) == 0 ? n/2 : n/3
    return x^2
end

In [ ]:
inestable(1), inestable(2)

In [ ]:
estable(1), estable(2)

Para compararlas, corremos muchas veces la funciones

In [ ]:
@time begin
    x = inestable(1)
    for i = 1:10000000
        x = inestable(i)
    end
    x
end

In [ ]:
@time begin
    x = estable(1)
    for i = 1:10000000
        x = estable(i)
    end
    x
end

**La función inestable es más de 10 veces más lenta que la función estable, justamente porque no preserva el tipo. Esto es, Julia no puede saber si el resultado será un racional o un entero.**

## <font color = "darkblue"> Definición de nuevos tipos </font>

Una de las partes atractivas de Julia es que permite definir nuevos tipos (o estructuras) que en algún sentido reflejan la abstracción de las componentes de un problema concreto. Para esto se utilizan las instrucciones `struct` y `mutable struct`.

Un tipo compuesto puede ser pensado como una estructura capaz de alojar dentro de ella diversos campos que pueden contener esencialmente cualquier objeto. En Julia existen dos clases de tipos compuestos: inmutables y mutables.

Los tipos inmutables se caracterizan por la propiedad de que al ser inicializados estos no pueden son modificados, contrario a lo que sucede con los mutables.




In [ ]:
struct TipoInmutable
    campo1
    campo2
end

Para un tipo mutable:

In [ ]:
mutable struct TipoMutable
    campo1
    campo2
end

**NOTA:** por cuestiones internas a Julia una vez dada la definición de un tipo, éste no puede ser reedefinido, por lo que si queremos actualizar la definición de alguno de nuestros tipos será necesario reiniciar el kernel.


Una vez dadas las definiciones de nuestro tipo podemos inicializarlo llamando al nombre del tipo y entre paréntesis los valores de los campos con los que fue definido

In [ ]:
TipoInmutable(1, 2)

In [ ]:
TipoInmutable("Hola", "Mundo")


Y podemos asignar esto a una variable de la manera usual

In [ ]:
tipo_inmutable = TipoInmutable("Soy ", "inmutable")
tipo_mutable = TipoMutable("Soy ", "mutable");


Podemos accesar a los campos del tipo mediante la sintáxis `tipo.campo`

In [ ]:
tipo_inmutable.campo1 * tipo_inmutable.campo2

In [ ]:
tipo_mutable.campo1 * tipo_mutable.campo2

Mediante esto es posible modificar los valores de los campos(para el caso de tipo mutables)

In [ ]:
tipo_mutable.campo1 = "Me "; tipo_mutable.campo2 = "han mutado!"
tipo_mutable.campo1 * tipo_mutable.campo2


Notemos que esto no es posible para el un tipo inmutable

In [ ]:
tipo_inmutable.campo1 = "Nuevo "

**NOTA:** En el caso de los tipos mutables, aunque dos objetos tengan los mismos valores en sus campos, **éstos no son iguales**, lo cual podría causar algunas confusiones, veamos

In [ ]:
objeto1 = TipoMutable(1, 1)

In [ ]:
objeto2 = TipoMutable(1, 1)

In [ ]:
objeto1 == objeto2


Además notemos

In [ ]:
objeto2 = objeto1

In [ ]:
objeto1 == objeto2

En este caso sí se trata del mismo objeto, es decir las variables objeto1 y objeto2 "apuntan" hacía el mismo lugar, por lo que la modificación de uno de sus campos afecta a ambas variables

In [ ]:
objeto1.campo1 = 0
objeto1

In [ ]:
objeto2

En ocasiones podemos no conocer los nombres de los campos de un tipo dado, es posible ver cuáles son mediante la función `fieldnames`

In [ ]:
fieldnames(TipoInmutable)

**Por ejemplo, tal vez necesitamos que nuestro tipo tenga campos cuyos únicos valores sean números, podemos lograr esto mediante la siguiente definición:**

In [ ]:
struct TipoNumero
    campo1::Number
    campo2::Number
end

De esta forma los únicos valores permitidos serán números, y en caso de querer construir un objeto que no satisface esta definición obtenemos un error

In [ ]:
TipoNumero(1, 1)

In [ ]:
TipoNumero("Hola", "mundo")

## <font color = "darkblue"> Tipos Paramétricos</font>


Un tipo puede estar definido mediante un parámetro, esto son los llamados tipos paramétricos, la definición de éstos es muy similar a lo visto anteriormente

In [ ]:
struct TipoParam{T}
    campo1::T
    campo2::T
end


Es decir, un tipo así definido implica una definición de TipoParam para cada posible valor de T, notemos que ambos campos han de pertenecer al mismo tipo, al crear el objeto Julia sabe determinar quién es T

In [ ]:
TipoParam(1, 1)

In [ ]:
TipoParam(1., 1.)

Si se intenta construir un objeto con campos de distinto tipo se obtendrá un error

In [ ]:
TipoParam(1, 1.)

Esto se puede solucionar mediante la introducción de más parámetros en la definición del tipo

In [ ]:
struct Tipo2Params{S, T}
    campo1::S
    campo2::T
end

In [ ]:
Tipo2Params(1, "s")

Los tipos paramétricos pueden ser restringidos a un subconjunto de todos los tipos disponible en Julia, para esto se hace uso de la notación `T <: Tipo` donde `T` es el parámetro, este enunciado se lee: `T` es un subtipo de `Tipo`, a diferencia de lo visto en la primer parte de este notebook, en este contexto esta sintáxis representa una afirmación.

In [ ]:
struct ParamRestr{T <: Real}
    campo::T
end

In [ ]:
ParamRestr(2.)

In [ ]:
ParamRestr(2 + 2im)

Los tipos paramétricos son una manera conveniente de definir objetos de cualquier clase obteniendo un mejor rendimento que empleando definiciones sin especificar el tipo de los campos(como las primeras definiciones que se vieron al principio de este notebook), esto se debe a que el compilador de Julia es capaz de determinar los valores que tendrán los campos del tipo con sólo ver la definición de éste, puede leerse más acerca de esto en la sección de ["Performance Tips"](https://docs.julialang.org/en/v1/manual/performance-tips/#Type-declarations-1) de la documentación de Julia.

In [ ]:
struct Cubo
    largo
    ancho
    alto
end
volumen(c::Cubo) = c.largo * c.ancho * c.alto

struct Cubo2
    largo::Float64
    ancho::Float64
    alto::Float64
end
volumen(c::Cubo2) = c.largo * c.ancho * c.alto

struct Cubo3{T <: Real}
    largo::T
    ancho::T
    alto::T
end

volumen(c::Cubo3) = c.largo * c.ancho * c.alto

In [ ]:
c1 = Cubo(2.1, 2.2, 2.3)
c2 = Cubo2(2.1, 2.2, 2.3)
c3 = Cubo3(2.1, 2.2, 2.3)

In [ ]:
volumen(c1) == volumen(c2) == volumen(c3)

In [ ]:
#using Pkg

In [ ]:
#import Pkg
#Pkg.add("BenchmarkTools")

In [ ]:
using BenchmarkTools

In [ ]:
@btime volumen(c1)

In [ ]:
@btime volumen(c2)

**En general, ser más específico respecto a los valores que pueden tomar los campos de nuestros tipos se traduce en que el compilador de Julia pueda generar código más eficiente.**

## <font color = "darkblue"> Métodos</font>

En Julia para una función dada pueden existir diversas definiciones(métodos) las cuales se aplican dependiendo del tipo de argumentos con las que la función es llamada, a esto se le conoce como *multiple dispatch*, esto permite definir comportamientos distintos de nuestra función para diversos casos, ya hemos visto ejemplos de esto, por ejemplo con operaciones elementales

Podemos ver cuáles son los métodos de una función utilizando la función `methods`

In [ ]:
methods(+)

Siempre será más útil dar explícitamente el tipo de los argumentos de la función mediante la sintáxis

```julia
function mi_func(x::TipoA, y::TipoB)
    ...
end
```


In [ ]:
f3(x::Int64, y::Int64) = (x^2, y^2)
g3(x::Int64, y::Int64) = (x +y, x- y)

In [ ]:
g3(3,3) + f3(2 ,2)

**En varias aplicaciones será útil poder escribir métodos de funciones presentes en Julia base para nuestros propios tipos, para poder hacer esto es necesario importar las funciones que queremos "extender" y posteriormente escribir el método correspondiente a nuestro tipo, veamos un ejemplo de esto creando la suma de funciones de la celda anterior.**

Para poder *extender* los operadores `+`, `-`, `*`, `/` y `^`, primero debemos importarlas:

In [ ]:
# Importamos la definición de los siguientes operadores
import Base: +, -, *, /, ^

**Con esta definición, podemos extender las operaciones aritméticas al igual que las funciones elementales.**

Y ahora las extendemos a conveniencia:

In [ ]:
+(f3::Tuple, g3::Tuple) = f3 .+  g3

In [ ]:
f3(x::Int64, y::Int64) = (x^2, y^2)
g3(x::Int64, y::Int64) = (x +y, x- y)

In [ ]:
g3(3,3) + f3(2 ,2)

**Otro ejemplo: Números Complejos**

In [ ]:
struct Complejo{S, T}
    a::S
    b::T
end

In [ ]:
+(c1::Complejo, c2::Complejo) = Complejo(c1.a + c2.a, c1.b + c2.b)
-(c1::Complejo, c2::Complejo) = Complejo(c1.a - c2.a, c1.b - c2.b)
*(c1::Complejo, c2::Complejo) = Complejo((c1.a * c2.a) - (c1.b * c2.b), (c1.a * c2.b) + (c1.b * c2.a)) 

In [ ]:
c1 = Complejo(1., 2)
c2 = Complejo(2, 2.)

In [ ]:
c1 + c2

In [ ]:
c1 - c2